In [ ]:
input_default_json_conf_file='cnn_emotion4_testing_holdout_default.json';

In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import json

In [1]:
import sys
sys.path.append('../library');

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
## Load json conf file
fd = open(os.path.join('./',input_default_json_conf_file));
DATA = json.load(fd);
fd.close()

# Variable globales

In [3]:
## Seed for the random variables
seed_number=0;

## Dataset 
dataset_base_dir     = DATA['dataset_base_dir'];
dataset_labels_file  = DATA['dataset_labels_file'];
dataset_name         = DATA['dataset_name'];

## Training hyperparameters
BATCH_SIZE = DATA['batch_size'];

## Model of network
## 'mobilenet_v3', 'efficientnet_b3', 'inception_v3', 'inception_resnet_v2', 'resnet_v2_50'
model_type = DATA['model_type'];

## Model filepath
model_dir = DATA['model_dir'];

## times to count time
times = DATA['times'];

## Output
output_base_dir = DATA['output_base_dir'];

##############################################

best_model_file = os.path.join(model_dir,'model_'+model_type+'.h5');


'''
print('   dataset_base_dir:',dataset_base_dir)
print('dataset_labels_file:',dataset_labels_file)
print('       dataset_name:',dataset_name)
print('         model_type:',model_type)
print('          model_dir:',model_dir)
print('              times:',times)
print('         BATCH_SIZE:',BATCH_SIZE)
print('    output_base_dir:',output_base_dir)
'''

   dataset_base_dir: /media/fernando/INFORMATION/TMP/DATABASE/RAW/DATASET1-FACE
dataset_labels_file: test.csv
       dataset_name: DATASET1-FACE
         model_type: efficientnet_b3
         model_file: /media/fernando/INFORMATION/DOCTORADO2/cnn_emotion4/cross-validation/DATASET1-FACE/skfold5_efficientnet_b3/model_1.h5
              times: 10
         BATCH_SIZE: 32
    output_base_dir: /mnt/boveda/DOCTORADO2/cnn_emotion4/test_holdout


# If command line

In [4]:
for n in range(len(sys.argv)):
    if   sys.argv[n]=='--dataset-dir':
        dataset_base_dir=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-file':
        dataset_labels_file=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-name':
        dataset_name=sys.argv[n+1];
    elif sys.argv[n]=='--model':
        model_type=sys.argv[n+1];
    elif sys.argv[n]=='--model-dir':
        model_dir=sys.argv[n+1];
    elif sys.argv[n]=='--times':
        times=int(sys.argv[n+1]);
    elif sys.argv[n]=='--batch-size':
        BATCH_SIZE=int(sys.argv[n+1]);
    elif sys.argv[n]=='--output-dir':
        output_base_dir=sys.argv[n+1];

best_model_file = os.path.join(model_dir,'model_'+model_type+'.h5');

print('   dataset_base_dir:',dataset_base_dir)
print('dataset_labels_file:',dataset_labels_file)
print('       dataset_name:',dataset_name)
print('         model_type:',model_type)
print('          model_dir:',model_dir)
print('              times:',times)
print('         BATCH_SIZE:',BATCH_SIZE)
print('    output_base_dir:',output_base_dir)

print('         model_file:',best_model_file)

   dataset_base_dir: /media/fernando/INFORMATION/TMP/DATABASE/RAW/DATASET1-FACE
dataset_labels_file: test.csv
       dataset_name: DATASET1-FACE
         model_type: efficientnet_b3
         model_file: /media/fernando/INFORMATION/DOCTORADO2/cnn_emotion4/cross-validation/DATASET1-FACE/skfold5_efficientnet_b3/model_1.h5
              times: 10
         BATCH_SIZE: 32
    output_base_dir: /mnt/boveda/DOCTORADO2/cnn_emotion4/test_holdout


# Set seed of random variables


In [5]:
np.random.seed(seed_number)
tf.keras.utils.set_random_seed(seed_number);

# Loading data of dataset

In [6]:
# Load filenames and labels
test_data = pd.read_csv(os.path.join(dataset_base_dir,dataset_labels_file));
print(test_data)

                                 filename     label
0      neutral/frame_count100_cam1_p0.png   neutral
1      negative/frame_count54_cam2_p0.png  negative
2         pain/frame_count317_cam2_p0.png      pain
3      neutral/frame_count359_cam1_p0.png   neutral
4       neutral/frame_count80_cam2_p0.png   neutral
...                                   ...       ...
1740   neutral/frame_count314_cam1_p0.png   neutral
1741  positive/frame_count378_cam1_p0.png  positive
1742       pain/frame_count56_cam2_p0.png      pain
1743  negative/frame_count347_cam1_p0.png  negative
1744       pain/frame_count53_cam0_p0.png      pain

[1745 rows x 2 columns]


# Data augmentation configuration

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

idg_test= ImageDataGenerator(rescale=1./255 )


# Creating output directory

In [8]:

output_dir = os.path.join(output_base_dir,dataset_name,'test_holdout',model_type);

os.makedirs(output_dir,exist_ok = True);

# Create new model

In [9]:
import BodyEmotion4Lib.lib_model as mpp

model, target_size = mpp.create_model(model_type=model_type,load_weights=False,file_of_weight=best_model_file);
model.summary()

mpp.save_model_parameters(model, os.path.join(output_dir,'parameters_stats.m'));


Loaded layer with efficientnet_b3
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1536)              10783528  
                                                                 
 dense (Dense)               (None, 4)                 6148      
                                                                 
Total params: 10,789,676
Trainable params: 6,148
Non-trainable params: 10,783,528
_________________________________________________________________


# Defining directories

In [10]:

test_data_generator  = idg_test.flow_from_dataframe(test_data, 
                                                    directory = dataset_base_dir,
                                                    target_size=target_size,
                                                    x_col = "filename", 
                                                    y_col = "label",
                                                    batch_size=BATCH_SIZE,
                                                    class_mode="categorical",
                                                    shuffle=True
                                                    );

Found 1745 validated image filenames belonging to 4 classes.


# Compile


In [11]:
# COMPILE NEW MODEL
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])


# Time

In [12]:
from pytictoc import TicToc
t = TicToc() 

# Evaluate best model

In [13]:
L=test_data_generator.samples;
print('L:',L);

t.tic();
for m in range(times):
    results = model.evaluate(test_data_generator,verbose=0)
t0=t.tocvalue();

results = dict(zip(model.metrics_names,results))
results['block_delayms']=t0*1000.0/(times*L);
print(results,"\n\n");


with open(os.path.join(output_dir,"results_testing.m"), 'w') as f: 
    for key, value in results.items(): 
        f.write('%s=%s;\n' % (key, value));


# final all json
with open(os.path.join(output_dir,"testing_data_results.json"), 'w') as f:
    json.dump(results, f,indent=4);

tf.keras.backend.clear_session()


L: 1745
{'loss': 0.24139080941677094, 'accuracy': 0.921489953994751} 


